In [1]:
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from tqdm import tqdm
from scipy.ndimage.measurements import label

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline  
%config InlineBackend.figure_format = 'retina'

In [2]:
class CFClassifier:
    def __init__(self, unpickle=False):
        
        self.file_name = "CFClassifier.pkl"
        
        if unpickle == True:
            with open(self.file_name, 'rb') as file:
                items = pickle.load(file)
                self.pix_per_cell = items["pix_per_cell"]
                self.cell_per_block = items["cell_per_block"]
                self.orient = items["orient"]
                self.hog_channel = items["hog_channel"]
                self.spatial_size = items["spatial_size"]
                self.nbins = items["nbins"]
                self.scaler = items["scaler"]
                self.svc = items["svc"]
        else:
            self.pix_per_cell = 8
            self.cell_per_block = 2
            self.orient = 9
            self.hog_channel = "ALL"
            self.spatial_size = (32, 32)
            self.nbins = 32
            self.scaler = None
            self.svc = None
        

    def train(self, training_images, visualise=False):

        features_car = self.__process_images(training_images[0], visualise=visualise)
        print(len(features_car))
        features_non_car = self.__process_images(training_images[1], visualise=visualise)
        print(len(features_non_car))
        
        X = np.vstack((features_car, features_non_car)).astype(np.float64) 
        print(X.shape)
        y = np.hstack((np.ones(len(features_car)), np.zeros(len(features_non_car))))

        self.scaler = StandardScaler()
        self.scaler.fit(X)
        scaledX = self.scaler.transform(X)
        
        X_train, X_test, y_train, y_test = train_test_split(scaledX, y, 
                                                            test_size=0.2, 
                                                            random_state=np.random.randint(0, 100))   
        
        self.svc = LinearSVC()
        self.svc.fit(X_train, y_train)
        score = round(self.svc.score(X_test, y_test), 4)
        
        with open(self.file_name, 'wb') as file:            
            items = {
                "pix_per_cell" : self.pix_per_cell,
                "cell_per_block": self.cell_per_block,
                "orient": self.orient,
                "hog_channel": self.hog_channel,
                "spatial_size": self.spatial_size,
                "nbins": self.nbins,
                "scaler": self.scaler,
                "svc": self.svc
            }
            pickle.dump(items, file)
        
        return score
        
    
    def test(self, test_images, test_labels):
        if self.svc == None:
            raise ValueError('Classifier is not initialized or ready')
        
        features = self.__process_images(test_images)
        scaledX = self.scaler.transform(features)        
        score = round(self.svc.score(scaledX, test_labels), 4)
        return score

    
    def predict(self, img, visualise=False):
        if self.svc == None:
            raise ValueError('Classifier is not initialized or ready')
                
        features = self.process_image(img, visualise=visualise)
        scaledX = self.scaler.transform(features.reshape(1, -1))        
        return self.svc.predict(scaledX)
    
    def process_image(self, img, visualise=False):

        # - Get spatial features
        spatial_features = self.get_spatial_features(img, size=self.spatial_size)
#         if visualise == True:
#             plt.subplot(1,5,4).plot(spatial_features)

        # - Get Hist features
        hist_features = self.get_colorhist_features(img, nbins=self.nbins)  
#         if visualise == True:
#             plt.subplot(1,5,5).plot(hist_features)

        # - Get HOG features
        hog_features = []        
        if self.hog_channel == "ALL":
            for channel in range(img.shape[2]):
                channel_hog_features, hog_image = self.get_hog_features(img[:,:,channel], 
                                                                   self.orient,
                                                                   self.pix_per_cell,
                                                                   self.cell_per_block,
                                                                   vis = visualise)                                
                hog_features.append(channel_hog_features)

                if visualise == True:
                    plt.subplot(1,5,channel+1).imshow(hog_image, cmap="gray")
        else:
            channel_hog_features, hog_image = self.get_hog_features(img[:,:,self.hog_channel], 
                                                               self.orient,
                                                               self.pix_per_cell,
                                                               self.cell_per_block,
                                                               vis = visualise)                                
            hog_features.append(channel_hog_features)

            if visualise == True:
                plt.subplot(1,5,1).imshow(hog_image, cmap="gray")
            
        
        hog_features = np.ravel(hog_features)

        return np.hstack((spatial_features, hist_features, hog_features))
        
    # ---------------------------------------------------------------------------
    # Private methods
    # ---------------------------------------------------------------------------
    def __process_images(self, images, visualise = False):
        
        all_features = []
        for file in tqdm(images):            
            img = cv2.imread(file)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)            
            img = img.astype(np.float32)/255
            features = self.process_image(img, visualise=visualise)   
            all_features.append(features)
            
        return all_features
            

    def get_spatial_features(self, img, size=(32, 32)):
        color1 = cv2.resize(img[:,:,0], size).ravel()
        color2 = cv2.resize(img[:,:,1], size).ravel()
        color3 = cv2.resize(img[:,:,2], size).ravel()
        return np.hstack((color1, color2, color3))
                       

    def get_colorhist_features(self, img, nbins=32):
        channel1_hist = np.histogram(img[:,:,0], bins=nbins)
        channel2_hist = np.histogram(img[:,:,1], bins=nbins)
        channel3_hist = np.histogram(img[:,:,2], bins=nbins)
        
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))        
        return hist_features    
    
    
    def get_hog_features(self, img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False):
        hog_image = None
            
        if vis == True:
            features, hog_image = hog(img, orientations=orient, 
                                      pixels_per_cell=(pix_per_cell, pix_per_cell),
                                      cells_per_block=(cell_per_block, cell_per_block), 
                                      transform_sqrt=True, 
                                      visualise=vis, feature_vector=feature_vec)
        else:
            features = hog(img, orientations=orient, 
                                      pixels_per_cell=(pix_per_cell, pix_per_cell),
                                      cells_per_block=(cell_per_block, cell_per_block), 
                                      transform_sqrt=True, 
                                      visualise=vis, feature_vector=feature_vec)
                    
        return features, hog_image
    


In [3]:
class CFCarFinder:
    def __init__(self, clf = None):
        self.clf = clf
    
    def findcar(self, img, draw_boxes=False):
        img_cpy = np.copy(img)
        shape = img_cpy.shape
        #img_cpy = cv2.resize(img_cpy, (np.int(shape[1]/1.5), np.int(shape[0]/1.5)))        
        hot_windows = []
        for w_size in range(64, 200, 64):
            windows = self.__get_windows(img_cpy, y_start=img_cpy.shape[0]//2, y_stop=img_cpy.shape[0]-100, xy_overlap=0.25, size_wh=(w_size,w_size))
            for window in windows:
                window_image = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
                window_image = cv2.cvtColor(window_image, cv2.COLOR_RGB2YCrCb)
                window_image = window_image.astype(np.float32)/255
                if self.clf.predict(window_image) == 1.:
                    hot_windows.append(window)

        if draw_boxes == True:
            for window in hot_windows:
                cv2.rectangle(img_cpy, window[0], window[1], (0,255,0), thickness=5)
            plt.imshow(img_cpy)
    
    def findcar_with_subsampling(self, img, scale=1.0):
                
        #make a copy
        draw_img = np.copy(img)
        
        #scale to 0-1
        img = img.astype(np.float32)/255
        
        ystart = img.shape[0]//2
        ystop = img.shape[0]-100
        
        #half of the iamge
        img_tosearch = img[ystart:ystop,:,:]        
        ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)

        if scale != 1:
            imshape = ctrans_tosearch.shape
            ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
            
        ch1 = ctrans_tosearch[:,:,0]
        ch2 = ctrans_tosearch[:,:,1]
        ch3 = ctrans_tosearch[:,:,2]

        # Define blocks and steps as above
        nxblocks = (ch1.shape[1] // clf.pix_per_cell) - clf.cell_per_block + 1
        nyblocks = (ch1.shape[0] // clf.pix_per_cell) - clf.cell_per_block + 1 
        nfeat_per_block = clf.orient*clf.cell_per_block**2
    
        # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
        window = 64
        nblocks_per_window = (window // clf.pix_per_cell) - clf.cell_per_block + 1
        cells_per_step = 1  # Instead of overlap, define how many cells to step
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
        # Compute individual channel HOG features for the entire image        
        hog1, hog_image = clf.get_hog_features(ch1, clf.orient, clf.pix_per_cell, clf.cell_per_block, feature_vec=False)
        hog2, hog_image = clf.get_hog_features(ch2, clf.orient, clf.pix_per_cell, clf.cell_per_block, feature_vec=False)
        hog3, hog_image = clf.get_hog_features(ch3, clf.orient, clf.pix_per_cell, clf.cell_per_block, feature_vec=False)
    
        hot_windows = []
        
        for xb in range(nxsteps):
            for yb in range(nysteps):
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                
                # Extract HOG for this patch
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

                xleft = xpos*clf.pix_per_cell
                ytop = ypos*clf.pix_per_cell

                # Extract the image patch
                subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
                # Get color features
                
                spatial_features = clf.get_spatial_features(subimg, size=clf.spatial_size)                
                hist_features = clf.get_colorhist_features(subimg, nbins=clf.nbins)

                # Scale features and make a prediction
                test_features = clf.scaler.transform(np.hstack((spatial_features, 
                                                                hist_features, hog_features)).reshape(1, -1))    
                test_prediction = clf.svc.predict(test_features)
            
                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    hot_windows.append([(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)])
                    #cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)         

        return hot_windows
    
    def get_labels(self, img, hot_windows, threshold=10):
        heat = np.zeros_like(img[:,:,0]).astype(np.float)        
        heat = self.add_heat(heat,hot_windows)            
        heat = self.apply_threshold(heat,threshold)      
        heatmap = np.clip(heat, 0, 255)                
        return label(heatmap)
        
    def __get_windows(self, img, x_start = None, x_stop=None, y_start=None, y_stop=None, xy_overlap=0.5, size_wh=(200, 200)):
        
        windows =[]
        #print(img.shape)
        i_height, i_width, channels = img.shape
        if x_start == None:
            x_start = 0
        
        if y_start == None:
            y_start = 0
            
        if x_stop == None:
            x_stop = i_width - int(size_wh[0]*xy_overlap)
        
        if y_stop == None:
            y_stop = i_height - int(size_wh[1]*xy_overlap)
            
        for x in range(x_start, x_stop, int(size_wh[0]*xy_overlap)):
            for y in range (y_start, y_stop, int(size_wh[1]*xy_overlap)):
                windows.append([(x, y),(x+size_wh[0], y+size_wh[1])])
                
        return windows
        

    def draw_labeles(self, img, labels):
        # Iterate through all detected cars
        for car_number in range(1, labels[1]+1):
            # Find pixels with each car_number label value
            nonzero = (labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Define a bounding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
        # Return the image
        return img        

    def get_boxes(self, labels):
        boxes = []
        for car_number in range(1, labels[1]+1):
            # Find pixels with each car_number label value
            nonzero = (labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Define a bounding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            boxes.append(bbox)

        return boxes
    
    def add_heat(self, heatmap, bbox_list):
        # Iterate through list of bboxes
        for box in bbox_list:
            # Add += 1 for all pixels inside each bbox
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
            
        return heatmap


    def apply_threshold(self, heatmap, threshold):
        # Zero out pixels below the threshold
        heatmap[heatmap <= threshold] = 0
        # Return thresholded map
        return heatmap


In [4]:
class Car:
    def __init__(self,position):
        self.position = position
        self.new_postion = None
        self.count = 0
        self.frame = 1
        self.flag = False
        self.long_count = 0
        self.postion_average = []
        
    def update(self,temp_position):
        if abs(temp_position[2]-self.position[2]) < 100 and abs(temp_position[3]-self.position[3]) < 100:
            if self.long_count > 2:
                self.postion_average.pop(0)
                self.postion_average.append(temp_position)
                self.new_postion = np.mean(np.array(self.postion_average), axis=0).astype(int)
                self.position = self.new_postion
                self.frame = 1
                self.count += 1

                return False

            self.position = temp_position
            self.postion_average.append(temp_position)
            self.count+=1

            return False        

    def get_position(self):
        self.frame+=1
        if self.count == 7 and self.long_count < 3 :
            self.new_postion = np.mean(np.array(self.postion_average), axis=0).astype(int)
            self.count = 0
            self.frame = 1
            self.long_count += 1
            if self.long_count < 2:
                self.postion_average = []

        if self.frame > 10:
            self.flag = True

        return self.new_postion, self.flag


In [ ]:
import glob
train = False


#Step 1. Train the model
clf = CFClassifier()

clf.spatial_size = (32, 32)
clf.nbins = 32
clf.hog_channels = 

car_images = glob.glob("../trainingdata/vehicles/*/*.png")
noncar_images = glob.glob("../trainingdata/non-vehicles/*/*.png")

training_data = (car_images, noncar_images)

score = clf.train(training_data, visualise=False)
print("Training score ", score)


In [ ]:
#Step 2. Finder
clf = CFClassifier(unpickle=True)
finder = CFCarFinder(clf)
testImage = cv2.cvtColor(cv2.imread("./test_images/test5.jpg"), cv2.COLOR_BGR2RGB)

# testImage = cv2.cvtColor(cv2.imread("./car_notcar/image16.png"), cv2.COLOR_BGR2YCrCb)
# clf.process_image(testImage, visualise=True)

#finder.findcar(testImage, draw_boxes=True)
boxes = finder.findcar_with_subsampling(testImage, scale=1.0)
labels = finder.get_labels(testImage, boxes, threshold = 12)
#print(labels)
finder.draw_labeles(testImage, labels)
# for box in boxes:
#     (x1, y1, x2, y2) = box[0][0], box[0][1], box[1][0], box[1][1]
#     cv2.rectangle(testImage, (x1, y1), (x2, y2), (0, 0, 255), thickness=6)
    
plt.imshow(testImage)
#plt.subplot(122).imshow(heatmap)

In [8]:
#Step 3. Video Processing

clf = CFClassifier(unpickle=True)
finder = CFCarFinder(clf)

counter = 0
labels = None
cars = []
hot_windows=[]

# Takes in a list of calculated centroids calculated from current frame from your own code  (both good and bad)

def process_video_frame2(img):
    global cars
#     global counter
#     global labels
   
#     if (counter % 5) != 0:
#         counter+=1
#         return img
    
#     counter+=1
    boxes = finder.get_boxes(finder.findcar_with_subsampling(img, scale=1))
    
    # Deal with car tracking
    for bbox in boxes:
        centroid = (bbox[0][0],bbox[0][1], bbox[1][0], bbox[1][1])
        new = True
        for car in cars:
            new = car.update(centroid)
            if new == False:
                break
        if new == True:
            cars.append(Car(centroid))

    next_cars = []
    positions = []

    for car in cars:
        position, flag = car.get_position()
        if flag == False:
            next_cars.append(car)
        positions.append(position)

    cars = next_cars

    # Outputs current relevant positions.

    try:
        for (x1, y1, x2, y2) in positions:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
    except:
        pass
    
    return img

def process_video_frame(img):
    global counter
    global hot_windows
    global labels

    #if (counter % 3) == 0:
    windows = finder.findcar_with_subsampling(img)
    hot_windows.extend(windows)
        
    if (counter % 10) == 0:
        labels = finder.get_labels(img, hot_windows, threshold=120)
        hot_windows=[]
        
    counter+=1

    if labels is not None and counter >= 10:
        return finder.draw_labeles(img, labels)
    else:
        return img
            

video_out = 'project_out.mp4'
clip = VideoFileClip("project_video.mp4")
out_clip = clip.fl_image(process_video_frame)
%time out_clip.write_videofile(video_out, audio=False)

[MoviePy] >>>> Building video project_out.mp4
[MoviePy] Writing video project_out.mp4


100%|█████████▉| 1260/1261 [44:38<00:02,  2.12s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_out.mp4 

CPU times: user 44min 18s, sys: 30.4 s, total: 44min 49s
Wall time: 44min 38s


In [ ]:
count = 1
clf = CFClassifier(unpickle=True)
finder = CFCarFinder(clf)

def processthis(img):
    global count
    cv2.imwrite("./output_images/{0}frame.png".format(count), cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    count+=1
    return img

video = VideoFileClip("project_video.mp4")
img = video.get_frame(40)

boxes = finder.findcar_with_subsampling(img, scale=1)
labels = finder.get_labels(img, boxes, threshold = 24)
finder.draw_labeles(img, labels)

plt.imshow(img)